### class 분류
#### 하루 주가 변동 분류 (4 분류)
  - 1: 오전 max 오전 min, 2: 오전 max 오후 min, 3: 오전 min, 오후 max, 4: 오후 man,  오후 min

#### 전일대비 +, - (2 분류)  -> 이 로직은 combine_features.ipynb 에서 처리함
  - 0 : <=0, 1: > 0

In [7]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
def read_pickle(com_name, date_range_ts):
    class_val = []
    for f_date in date_range_ts :
        f_name = '{}/{}_10min_price_{}.pkl'.format(pkl_directory, com_name, f_date.strftime('%Y%m%d'))
        try : 
            df = pd.read_pickle(f_name)
            class_num = classify_into_4(df)
            d_c_temp = [f_date.strftime('%Y-%m-%d'), class_num]
            class_val.append(d_c_temp)
        except :
            continue
    return class_val

In [3]:
def classify_into_4(df):
    df_new = df[(df['time'].astype(str) > '09:00:00') & (df['time'].astype(str) < '15:30:00')]
    #  9시 30분(미포함) 이전 제외, 오후 3시 30분(포함) 이후 제외 후 최고가 시간, 최저가 시간 비교
#     df_c = df_new['time'].astype(str).copy() #timestampe에서 str로 변환 (크기 비교를 하기 위하여)
#     df_new['time'] =  df_c
    time_max = str(df_new[df_new['price'] == df_new['price'].max()]['time'].iloc[0]) # str() change datetime.time to string
    time_min = str(df_new[df_new['price'] == df_new['price'].min()]['time'].iloc[0])
    time_criteria = '12:00:00'
    if (time_max <= time_criteria) & (time_min <= time_criteria):
        class_num = 1
    elif (time_max <= time_criteria) & (time_min >= time_criteria):
        class_num = 2
    elif (time_max >= time_criteria) & (time_min <= time_criteria):
        class_num = 3
    elif (time_max >= time_criteria) & (time_min >= time_criteria):
        class_num = 4
    # : 오전 max 오전 min, 2: 오전 max 오후 min, 3: 오전 min, 오후 max, 4: 오후 man,  오후 min
    return class_num

In [4]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [5]:
start_date = '20220620'
end_date = datetime.date.today() - datetime.timedelta(days = 1) # 오늘날짜 하루전까지
date_range_ts = pd.date_range(start=start_date, end=end_date)
pkl_directory = '../data/company_daily_price_pkl'

In [6]:
for key, val in code.items():
    class_val = read_pickle(val[1], date_range_ts)
    df = pd.DataFrame(class_val)
    df.columns = ['date', 'value']
    df.to_pickle(val[1]+'.pkl')

In [ ]:
## 아래 내용은 combine_features.ipynb에서 처리하는 것이 좋음
### historical data를 같이 처리할 수 있으므로

In [5]:
fname = '../data/company_pkl/sec_historical.pkl'
df = pd.read_pickle(fname)

In [6]:
df.head()

,date,open,high,low,close,close_cr,vol
0,2020-01-01,,,,,,
1,2020-01-02,55500.0,56000.0,55000.0,55200.0,-1.08,719663.0
2,2020-01-03,56000.0,56600.0,54900.0,55500.0,0.54,860207.0
3,2020-01-04,,,,,,
4,2020-01-05,,,,,,
